**Building Classification Model**

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=18b7157ddf0e500cd27237a5253154bf15bd10288df6a96fbf6a179ce83d8412
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


1.-Load Dataset

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('binary_class').getOrCreate()

In [3]:
df=spark.read.csv('classification_data.csv',inferSchema=True,header=True)

2.-Explore Data

In [4]:
print((df.count(),len(df.columns)))

(46751, 12)


In [5]:
df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- loan_purpose: string (nullable = true)
 |-- is_first_loan: integer (nullable = true)
 |-- total_credit_card_limit: integer (nullable = true)
 |-- avg_percentage_credit_card_limit_used_last_year: double (nullable = true)
 |-- saving_amount: integer (nullable = true)
 |-- checking_amount: integer (nullable = true)
 |-- is_employed: integer (nullable = true)
 |-- yearly_salary: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- dependent_number: integer (nullable = true)
 |-- label: integer (nullable = true)



In [6]:
df.show(5)

+-------+------------+-------------+-----------------------+-----------------------------------------------+-------------+---------------+-----------+-------------+---+----------------+-----+
|loan_id|loan_purpose|is_first_loan|total_credit_card_limit|avg_percentage_credit_card_limit_used_last_year|saving_amount|checking_amount|is_employed|yearly_salary|age|dependent_number|label|
+-------+------------+-------------+-----------------------+-----------------------------------------------+-------------+---------------+-----------+-------------+---+----------------+-----+
|    A_1|    personal|            1|                   7900|                                            0.8|         1103|           6393|          1|        16400| 42|               4|    0|
|    A_2|    personal|            0|                   3300|                                           0.29|         2588|            832|          1|        75500| 56|               1|    0|
|    A_3|    personal|            0|    

In [7]:
df.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|16201|
|    0|30550|
+-----+-----+



In [8]:
df.groupBy('loan_purpose').count().show()

+------------+-----+
|loan_purpose|count|
+------------+-----+
|      others| 6763|
|   emergency| 7562|
|    property|11388|
|  operations|10580|
|    personal|10458|
+------------+-----+



3.-Data Transformation

In [9]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

In [10]:
loan_purpose_indexer = StringIndexer(inputCol="loan_purpose", outputCol="loan_index").fit(df)

In [11]:
df = loan_purpose_indexer.transform(df)

In [12]:
loan_encoder = OneHotEncoder(inputCol="loan_index", outputCol="loan_purpose_vec")

In [13]:
ohe = loan_encoder.fit(df)

In [14]:
df = ohe.transform(df)

In [15]:
df.select(['loan_purpose','loan_index','loan_purpose_vec']).show(3,False)

+------------+----------+----------------+
|loan_purpose|loan_index|loan_purpose_vec|
+------------+----------+----------------+
|personal    |2.0       |(4,[2],[1.0])   |
|personal    |2.0       |(4,[2],[1.0])   |
|personal    |2.0       |(4,[2],[1.0])   |
+------------+----------+----------------+
only showing top 3 rows



In [16]:
from pyspark.ml.feature import VectorAssembler

In [17]:
df_assembler = VectorAssembler(inputCols=['is_first_loan', 'total_credit_card_limit', 'avg_percentage_credit_card_limit_used_last_year', 'saving_amount',
 'checking_amount',
 'is_employed',
 'yearly_salary',
 'age',
 'dependent_number',
 'loan_purpose_vec'], outputCol="features")

In [18]:
df = df_assembler.transform(df)

In [19]:
df.select(['features','label']).show(10,False)

+--------------------------------------------------------------------+-----+
|features                                                            |label|
+--------------------------------------------------------------------+-----+
|[1.0,7900.0,0.8,1103.0,6393.0,1.0,16400.0,42.0,4.0,0.0,0.0,1.0,0.0] |0    |
|[0.0,3300.0,0.29,2588.0,832.0,1.0,75500.0,56.0,1.0,0.0,0.0,1.0,0.0] |0    |
|[0.0,7600.0,0.9,1651.0,8868.0,1.0,59000.0,46.0,1.0,0.0,0.0,1.0,0.0] |0    |
|[1.0,3400.0,0.38,1269.0,6863.0,1.0,26000.0,55.0,8.0,0.0,0.0,1.0,0.0]|0    |
|[0.0,2600.0,0.89,1310.0,3423.0,1.0,9700.0,41.0,4.0,0.0,0.0,0.0,1.0] |1    |
|[0.0,7600.0,0.51,1040.0,2406.0,1.0,22900.0,52.0,0.0,0.0,1.0,0.0,0.0]|0    |
|[1.0,6900.0,0.82,2408.0,5556.0,1.0,34800.0,48.0,4.0,0.0,1.0,0.0,0.0]|0    |
|[0.0,5700.0,0.56,1933.0,4139.0,1.0,32500.0,64.0,2.0,0.0,0.0,1.0,0.0]|0    |
|[1.0,3400.0,0.95,3866.0,4131.0,1.0,13300.0,23.0,3.0,0.0,0.0,1.0,0.0]|0    |
|[0.0,2900.0,0.91,88.0,2725.0,1.0,21100.0,52.0,1.0,0.0,0.0,1.0,0.0]  |1    |

In [20]:
model_df=df.select(['features','label'])

4.-Splitting into Train and Test Data

In [21]:
training_df,test_df=model_df.randomSplit([0.75,0.25])

5.-Model Training


In [22]:
from pyspark.ml.classification import LogisticRegression

In [23]:
log_reg=LogisticRegression().fit(training_df)

In [24]:
lr_summary=log_reg.summary

In [25]:
lr_summary.accuracy

0.8951819579702717

In [26]:
lr_summary.areaUnderROC

0.9587580681268282

In [27]:
print(lr_summary.precisionByLabel)

[0.9252283105022832, 0.8397342408037595]


In [28]:
print(lr_summary.recallByLabel)

[0.9141902737408355, 0.8588713018977376]


In [29]:
predictions = log_reg.transform(test_df)

In [30]:
predictions.show(10)

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(13,[0,1,2,3,4,7]...|    0|[1.13134892472537...|[0.75608775245570...|       0.0|
|(13,[0,1,2,3,4,7]...|    0|[8.16335295613624...|[0.99971517563222...|       0.0|
|(13,[0,1,2,3,4,7]...|    0|[5.19817121188361...|[0.99450371383857...|       0.0|
|(13,[0,1,2,3,4,7]...|    1|[1.36273080363382...|[0.79620316640850...|       0.0|
|(13,[0,1,2,3,4,7,...|    1|[-6.2017895237770...|[0.00202170659190...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-6.5114980928908...|[0.00148404584623...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-6.5446272383021...|[0.00143575558887...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-5.0306067714176...|[0.00649241727520...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-6.2918094780836...|[0.00184798546107...|       1.0|
|(13,[0,1,2,3,4,

In [31]:
model_predictions = log_reg.transform(test_df)

In [32]:
model_predictions = log_reg.evaluate(test_df)

In [33]:
model_predictions.accuracy

0.890398005673515

In [34]:
model_predictions.areaUnderROC

0.9593707479010423

In [35]:
print(model_predictions.recallByLabel)

[0.9135884784637952, 0.8483309143686503]


In [36]:
print(model_predictions.precisionByLabel)

[0.9161540518855309, 0.844043321299639]


6.-Hyperparameter Tuning

In [37]:
from pyspark.ml.classification import RandomForestClassifier

In [38]:
rf = RandomForestClassifier()

In [39]:
rf_model = rf.fit(training_df)

In [40]:
model_predictions = rf_model.transform(test_df)

In [41]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [42]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [43]:
evaluator = BinaryClassificationEvaluator()

In [44]:
rf = RandomForestClassifier()

In [45]:
paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [5,10,20,25,30])
             .addGrid(rf.maxBins, [20,30,40 ])
             .addGrid(rf.numTrees, [5, 20,50])
             .build())


In [46]:
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

In [47]:
cv_model = cv.fit(training_df)

7.-Best Model

In [48]:
best_rf_model = cv_model.bestModel

In [49]:
model_predictions = best_rf_model.transform(test_df)

In [50]:
true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()


In [51]:
actual_pos=model_predictions.filter(model_predictions['label']==1).count()

In [52]:
pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()

In [53]:
recall_rate=float(true_pos)/(actual_pos)

In [54]:
print(recall_rate)

0.8981615868408321


In [55]:
float(true_pos)/(pred_pos)

0.8648963428837643